In [ ]:
from argparse import ArgumentParser
import os.path as op
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import socket
import pandas as pd
import numpy as np
import seaborn as sns
from time import sleep

In [ ]:
%matplotlib inline

In [ ]:
# Generate base names
class DataFinder:
    def __init__(self, basedir, subject, run):
        self.basedir = basedir
        self.projectdir = op.join(
            basedir,
            "NIMH_SFIM",
            "handwerkerd",
            "ComplexMultiEcho1",
            "Data",
        )
        self.subject = subject
        self.run = run

    def set_subject(self, subject: int):
        self.subject = subject

    def set_run(self, run: int):
        self.run = run

    def subid(self):
        return f"sub-{self.subject:02}"

    def runid(self):
        return f"run{self.run:02}"

    def regressor_dir(self):
        return op.join(
            self.projectdir,
            self.subid(),
            "Regressors",
            "RejectedComps_c75",
        )

    def mixing_dir(self):
        sd = self.subid()
        return op.join(
            self.projectdir,
            sd,
            "afniproc_orig",
            "WNW",
            f"{sd}.results",
            f"tedana_c75_r{self.run:02}"
        )

    def regressor_prefix(self):
        subject = self.subject
        run = self.run
        return op.join(
            self.regressor_dir(),
            f"{self.subid()}_r{run:02}_CombinedRejected_c75_"
        )

    def combined_metrics(self):
        return self.regressor_prefix() + "Combined_Metrics.csv"

    def combined_betas(self):
        return self.regressor_prefix() + "betas.csv"

    def combined_r2(self):
        return self.regressor_prefix() + "R2vals.csv"

    def combined_f(self):
        return self.regressor_prefix() + "Fvals.csv"

    def combined_p(self):
        return self.regressor_prefix() + "pvals.csv"

    def full_model(self):
        return self.regressor_prefix() + "FullRegressorModel.csv"

    def mixing_matrix(self):
        return op.join(
            self.mixing_dir(),
            "ica_mixing.tsv",
        )

In [ ]:
def plot_fit(basedir, outdir, subject, run, component, xloc=0.05, yloc=0.95):


    namer = DataFinder(basedir, subject, run)
    Y = np.asarray(pd.read_csv(namer.mixing_matrix(), sep='\t'))
    X_full = np.asarray(pd.read_csv(namer.full_model()))
    betas = pd.read_csv(namer.combined_betas())

    metrics = pd.read_csv(namer.combined_metrics())
    kappa = np.round(metrics["kappa"].iloc[component],1)
    rho = np.round(metrics["rho"].iloc[component], 1)
    varex = np.round(metrics["variance explained"].iloc[component], 1)
    signif_types = ['Motion', 'Phys_Freq', 'Phys_Variability', 'WM & CSF']
    signif_label = 'Signif'
    signif_gap = ':'
    for signif in signif_types:
        tmp = metrics[f"Signif {signif}"].iloc[component]
        if tmp:
            signif_label = f"{signif_label}{signif_gap} {signif}"
            signif_gap=','
    betas.drop(
        columns=[betas.columns[0], betas.columns[1]],
        axis=1,
        inplace=True,
    )

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ica = Y
    fit = np.asarray(np.matmul(X_full[:, 2:], betas.T))

    c = component

    ica_ts = ica[:, c]
    fit_ts = fit[:, c]
    ax.plot(ica_ts, color='black')
    ax.plot(fit_ts, color='red')
    textstr = '\n'.join((
        f"sub-{subject}, run {run}",
        f"kappa: {kappa}",
        f"rho: {rho}",
        f"var explained: {varex}",
        f"{signif_label}"
    ))
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.autoscale(enable=True, axis='y', tight=True)
    ax.text(xloc, yloc, textstr, transform=ax.transAxes, fontsize=14,font='Baskerville',
        verticalalignment='top')
    plt.tight_layout(pad=1.02)
    plt.savefig(f"{outdir}FitTS_sub-{subject}_run-{run}_comp-{component}.svg")

In [ ]:
if "biowulf" in socket.gethostname():
    basedir = "/data/"
    outdir = "/data/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data/GroupResults/"
else:
    basedir = "/Volumes/"
    outdir = "/Users/handwerkerd/Documents/Presentations/HBM2022/"


subject = 7
run = 3
component = 1

namer = DataFinder(basedir, subject, run)
df = pd.read_csv(namer.combined_metrics())
kappas = df["kappa"]
rhos = df["rho"]
varex = df["variance explained"]
Y = np.asarray(pd.read_csv(namer.mixing_matrix(), sep='\t'))
X_full = np.asarray(pd.read_csv(namer.full_model()))
betas = pd.read_csv(namer.combined_betas())

betas.drop(
    columns=[betas.columns[0], betas.columns[1]],
    axis=1,
    inplace=True,
)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ica = Y
fit = np.asarray(np.matmul(X_full[:, 2:], betas.T))

c = component

ica_ts = ica[:, c]
fit_ts = fit[:, c]
ax.plot(ica_ts, color='black')
ax.plot(fit_ts, color='red')

In [ ]:
def scatter_cme(ax, fname: str, to_plot) -> None:
    df = pd.read_csv(fname)
    kappas = df["kappa"]
    rhos = df["rho"]
    varex = df["variance explained"]
    size = np.sqrt(varex) * 20
    rej_both = np.logical_and(df["Tedana Rejected"] == True, df["Regressors Rejected"] == True)
    rej_tedonly = np.logical_and(df["Tedana Rejected"] == True, df["Regressors Rejected"] == False)
    rej_regonly = np.logical_and(df["Tedana Rejected"] == False, df["Regressors Rejected"] == True)
    acc_all = np.logical_and(df["Tedana Rejected"] == False, df["Regressors Rejected"] == False)
    colors = pd.Series(data=["none" for _ in kappas])
    if to_plot[0]:
        colors[acc_all] = "green"
    if to_plot[1]:
        colors[rej_both] = "red"
    if to_plot[2]:
        colors[rej_tedonly] = "orange"
    if to_plot[3]:
        colors[rej_regonly] = "brown"
        rej_regonly_idx = rej_regonly.index[rej_regonly == True].tolist()
        print(f"Reg only rejections (comp number and kappa): \n{kappas[rej_regonly_idx]}")
    ax.scatter(kappas, rhos, s=size, c=colors)

In [ ]:
fig = plt.figure(figsize=[10,10])
ax = fig.add_subplot(1, 1, 1)
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

subplots = [fig.add_subplot(2, 2, i + 1) for i in range(4)]
titles = ("Accepted All", "Rejected Both", "Non-Bold Only", "Motion/Phys Only")
top = [False for i in range(4)]
title_size = 24
label_size = 24
tick_size = 16
for s in (np.arange(13) + 1):
    for r in (np.arange(3) + 1):
        namer = DataFinder(basedir, s, r)
        print(f"Sub-{s}, run {r}")
        for p in range(4):
            subplots[p].set_title(titles[p], y=0.75, size=title_size, font='Baskerville')
            top[p] = True
            scatter_cme(subplots[p], namer.combined_metrics(), top)
            top[p] = False
            subplots[p].tick_params(labelsize=tick_size)

# Set common labels
ax.set_xlabel('kappa', size=label_size, font='Baskerville')
ax.set_ylabel('rho', size=label_size, font='Baskerville')
plt.savefig(f"{outdir}Combined_kappa_rho_plots.svg")

#### A list of subjects, runs, component numbers, and kappa values for regressor only rejected components with high kappa values
#### These were identified by eye using the output from the above cell
- Sub 1 run 1: 68, 108
- Sub 1 run 3: 73 167
- Sub 2 run 3: 71 125
- Sub 5 run 1: 29 120
- Sub 6 run 2: 38 99
- Sub 9 run 1: 70 102
- Sub 11 run 3: 12 111

- Get the ICA spatial components with the following commands on biowulf:

cd /data/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data/sub-06/afniproc_orig/WNW/sub-06.results/tedana_c75_r02<BR>
3drefit -view orig -space ORIG ica_components.nii.gz<BR>
afni ica_components.nii.gz ../anat_final.sub-??+orig.HEAD




In [ ]:
plot_fit(basedir, subject=1, run=1, component=68, outdir=outdir, xloc=0.55, yloc=0.4)
plot_fit(basedir, subject=1, run=3, component=73, outdir=outdir, xloc=0.48, yloc=0.32)
plot_fit(basedir, subject=2, run=3, component=71, outdir=outdir, xloc=0.1, yloc=0.96)
plot_fit(basedir, subject=5, run=1, component=29, outdir=outdir, xloc=0.3, yloc=0.3)
plot_fit(basedir, subject=6, run=2, component=38, outdir=outdir, xloc=0.55, yloc=0.3)
plot_fit(basedir, subject=9, run=3, component=12, outdir=outdir, xloc=0.1, yloc=0.96)
plot_fit(basedir, subject=11, run=3, component=12, outdir=outdir, xloc=0.1, yloc=0.4)

In [ ]:
# Make counts of significant fits per run
percent_signif = pd.DataFrame(columns=['Full Model','Motion Model','Phys_Freq Model','Phys_Variability Model','WM & CSF Model'],
                            index = np.arange(13*3))
reg_cat = ['Motion Model','Phys_Freq Model','Phys_Variability Model','WM & CSF Model']
idx=0
for s in (np.arange(13) + 1):
    for r in (np.arange(3) + 1):
        namer = DataFinder(basedir, s, r)
        pvals = pd.read_csv(namer.combined_p())
        numcomp = len(pvals)
        tmp_signif=pvals['Full Model']<(0.05/numcomp)
        percent_signif['Full Model'].iloc[idx] = 100*np.sum(tmp_signif)/numcomp
        for reg in reg_cat:
            percent_signif[reg].iloc[idx] = 100*np.sum((pvals[reg]<(0.05/numcomp)) * tmp_signif)/numcomp
        idx += 1
print(percent_signif)


In [ ]:
fig = plt.figure(figsize=(10,7))
for regidx, reg in enumerate(percent_signif.columns):
    plt.subplot(2,3,regidx+1)
    plt.hist(percent_signif[reg])
    plt.title(reg)
plt.xlabel("% of components with signif fit to regressors")

In [ ]:
# KDE rather than histogram version
# fig = plt.figure(figsize=(10,7))
# for regidx, reg in enumerate(percent_signif.columns):
# #     plt.subplot(2,3,regidx+1)
#     percent_signif[reg].plot.kde(bw_method=0.3, ind=50)
#  #   plt.title(reg)
# plt.xlabel("% of components with significant fit to regressors", fontsize=22, font='Baskerville')
# plt.xticks(font='Baskerville', fontsize=18)
# plt.yticks(font='Baskerville', fontsize=16)
# plt.ylabel("KDE density", fontsize=22, font='Baskerville')
# plt.legend(percent_signif.columns)
# plt.xlim([-1,101])
# plt.autoscale(enable=True, axis='y', tight=True)
# plt.savefig(f"{outdir}Regressor_Fit_Histograms.svg")

In [ ]:
font = font_manager.FontProperties(family='Baskerville',
                                   style='normal', size=16)
colorlist=['blue', 'orange', 'green', 'red', 'purple']
fig = plt.figure(figsize=(10,7))
linehand=['', '', '', '', '']
for regidx, reg in enumerate(percent_signif.columns):
    count, bins = np.histogram(percent_signif[reg], bins=np.linspace(0,100,20))
    bin_centers = 0.5*(bins[1:]+bins[:-1])
    linehand[regidx] = plt.plot(bin_centers,count, color=colorlist[regidx])
plt.legend(percent_signif.columns,prop=font)
for regidx, reg in enumerate(percent_signif.columns):
    plt.plot([np.mean(percent_signif[reg]), np.mean(percent_signif[reg])], [0, 4], color=colorlist[regidx], linestyle='dashed')
    print(f"{reg} mean: {np.mean(percent_signif[reg])}")
plt.xlabel("% of components with significant fit to regressors", fontsize=22, font='Baskerville')
plt.xticks(font='Baskerville', fontsize=20)
plt.yticks(font='Baskerville', fontsize=20)
plt.ylabel("# of runs", fontsize=24, font='Baskerville')
plt.xlim([0,100])
plt.autoscale(enable=True, axis='y', tight=True)
plt.tight_layout(pad=1.01)
plt.savefig(f"{outdir}Regressor_Fit_Histograms.svg")